In [1]:
#IPython is what you are using now to run the notebook
import IPython
print( "IPython version:      %6.6s (need at least 1.0)" % IPython.__version__)

# Numpy is a library for working with arrays and matrices
import numpy as np
print( "Numpy version:        %6.6s (need at least 1.7.1)" % np.__version__)

# SciPy implements many different numerical algorithms
import scipy as sp
print( "SciPy version:        %6.6s (need at least 0.12.0)" % sp.__version__)

# Pandas makes working with data tables easier
import pandas as pd
print( "Pandas version:       %6.6s (need at least 0.11.0)" % pd.__version__)

# Module for plotting
import matplotlib.pyplot as plt  
from pylab import *
print( "Mapltolib version:    %6.6s (need at least 1.2.1)" %
       matplotlib.__version__)
%matplotlib inline
# necessary for in-line graphics

# SciKit Learn implements several Machine Learning algorithms
import sklearn
print( "Scikit-Learn version: %6.6s (need at least 0.13.1)" %
       sklearn.__version__)
import os
# for certain system-related functions

IPython version:       7.8.0 (need at least 1.0)
Numpy version:        1.16.5 (need at least 1.7.1)
SciPy version:         1.3.1 (need at least 0.12.0)
Pandas version:       0.25.1 (need at least 0.11.0)
Mapltolib version:     3.1.1 (need at least 1.2.1)
Scikit-Learn version: 0.21.3 (need at least 0.13.1)


### 1 Explore and prepare the data (20pt)
As the first step, explore the data.

Q1. (2pt) Load the data. How many responses and variables do we have?

Answer 1:

The dataset World Values Survey data has 90350 responses (rows) and 328 variables (columns).

In [2]:
# Reading the world Value data
wvs_df = pd.read_csv('C:/Users/vaide/Documents/574-DS2/ps4/wvs.csv.bz2', sep ='\t')

#Exploring the data
print(wvs_df.head())
print("\n The nmber of rows and columns are : \n")

print(wvs_df.shape)

   V2  V4  V5  V6  V7  V8  V9  V10  V11  V12  ...  MN_228S8  MN_229A  MN_230A  \
0  12   1   1   1  -2   1   1    2    1    1  ...         3       -3       -3   
1  12   1   2   3   4   2   2    2    2    2  ...         3       -3       -3   
2  12   1   3   2   4   2   1    2    2    2  ...         4        1        1   
3  12   1   1   3   4   3   1    2    1    2  ...         2        2        1   
4  12   1   1   1   2   1   1    1    3    2  ...         2        2        1   

   MN_233A  MN_237B1  MN_249A1  MN_249A3  I_RELIGBEL  I_NORM1  I_VOICE1  
0       -3        -3         1         1         0.0      1.0      0.00  
1       -3        -3         2        -1         0.0      1.0      0.66  
2        2        -3         1         1         0.0      1.0      0.33  
3        2        -3         1         2         0.0      1.0      0.00  
4        2        -3         1         2         0.0      1.0      0.66  

[5 rows x 328 columns]

 The nmber of rows and columns are : 

(9035

Q2. (3pt) Create a summary table over all responses for V204 : is abortion justifiable. How many nonmissing responses (i.e. positive answers) do you find? Describe the the opinion about the abortion among the global pool of respondents.

Answer 2:
There are 85742 non-missing responses and 4608 missing responses for the V204 variable.

Opinion about abortion among the global pool of respondents is :

About 60% of the global pool of respondents think that abortion is not justifiable (rating 1-3).

About 10% of the global pool of respondents think that abortion is justifiable (rating 8-10).

About 5% of the global pool of respondents had given inapplicable/missing responses.

In [3]:
# Getting the count for both missing and non-misising responses
data = [["Non-Missing ", len(wvs_df[wvs_df.V204 > 0])],["Missing",len(wvs_df[wvs_df.V204 < 0])]]
data_pd = pd.DataFrame(data, columns=["Responses","Count"])
data_pd

,Responses,Count
0,Non-Missing,85742
1,Missing,4608


In [4]:
# Getting the count of each V204 response
abortion_summary = pd.DataFrame(wvs_df.groupby('V204').V204.count())
#Renaming the column
abortion_summary.columns = ["Count"]
# Creating the Percentage column
abortion_summary["Percentage_Opinion"] = abortion_summary["Count"] *100 / len(wvs_df)
abortion_summary


,Count,Percentage_Opinion
V204,,
-5,23,0.025457
-4,1523,1.685667
-2,1045,1.156613
-1,2017,2.232429
1,40227,44.523520
2,7896,8.739347
3,6294,6.966242
4,4497,4.977310
5,9580,10.603210


Q3. (4pt) Now remove missings. 

We do it in two ways:

(a) remove everything that are not positive integers for V204 and V2 (country).

(b) for all other variables, remove the missings in the sense of missing value on computer. 

You may leave negative answers in the data, otherwise I am afraid your sample size collapses. What is the final number of observations?

Answer 3:

The new dataset has 79267 observations after removing the missing data.

In [5]:
# Part (a)
#  Removing everything not positive for V204 
wvs_clean_df = wvs_df[wvs_df.V204 > 0]
#  Removing everything not positive for V2 
wvs_clean_df = wvs_clean_df[wvs_clean_df.V2 > 0]

# Part (b)

# Dropping all Nas
wvs_clean_df = wvs_clean_df.dropna()
wvs_clean_df.shape

(79267, 328)

Q4. (2pt) In order to simplify the analysis below, create a new binary variable abortion as abortion =

1: V204 > 3

0: otherwise

Answer 4:

We have created a new binary variable "abortion" as specified.

In [6]:
# Creating a new binary varibale abortion
wvs_clean_df['abortion'] = np.where(wvs_clean_df["V204"] > 3 , 1 , 0)
# Unique values of new variable abortion
wvs_clean_df['abortion'].unique()

array([0, 1], dtype=int64)

Q5. (5pt) Compute (pearson) correlation table between abortion and all other variables in the data. There are many of these!
Take a look at a few variables that have strong correlation with abortion. What do these represent?

Answer 5:

From the correlation table,we can see that apart from V204 i.e justification for abortion other variable V205 i.e is divorce jsutifiable? variable has the correlation 0.55 with our response variable abortion. It means that the people (respondents) who find abortion justifiable also feel that divorce is justifiable too and vice versa. The other variables have correlation less then 0.5 and thus have low correlation with our repsonse variable abortion. 

In [7]:
# Getting correlation of all predictors with our response variable abortion
wvs_abor_cor = pd.DataFrame(wvs_clean_df[wvs_clean_df.columns[1:]].corr()['abortion'][:])
# Sorting the correlation in descending order
wvs_abor_cor_sor = wvs_abor_cor.sort_values(by='abortion', ascending=False)
#Renaming the column
wvs_abor_cor_sor.columns = ["Correlation"]
wvs_abor_cor_sor

,Correlation
abortion,1.000000
V204,0.881048
V205,0.548653
V203,0.485419
V206,0.446394
...,...
V138,-0.142894
V255,-0.149844
V223,-0.165924
V252,-0.191483


Q6. (4pt) convert country code V2 into dummies. First rename V2 to country. Thereafter use pd.get_dummies along these lines. Afterwards, remove country variable from the data. How many rows/columns do you have now? How many country dummies does the data contain?

Answer 6: 

After converting the V2 variable to country dummies, the data has 79267 rows and 386 columns. After removing one dummy variable, the dataset will have 385 variables

Our dataset contains 58 contry dummy variables. We have removed one country dummy variable "country_12" in order to avoid perfect multicollinearity. Now the dataset has 57 variables country dummy variables

In [8]:
# Renaming V2 to country
wvs_clean_df.rename(columns={'V2':'country'} , inplace=True)
# Creating dummy varibales for country
wvs_country_df = pd.get_dummies(wvs_clean_df, columns = ['country'])

print(wvs_country_df.head())

print("\nThe number of rows and columns are : ")
print(wvs_country_df.shape)

   V4  V5  V6  V7  V8  V9  V10  V11  V12  V13  ...  country_752  country_764  \
0   1   1   1  -2   1   1    2    1    1    1  ...            0            0   
1   1   2   3   4   2   2    2    2    2    1  ...            0            0   
2   1   3   2   4   2   1    2    2    2    2  ...            0            0   
3   1   1   3   4   3   1    2    1    2    2  ...            0            0   
4   1   1   1   2   1   1    1    3    2    1  ...            0            0   

   country_780  country_788  country_792  country_804  country_840  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   country_858  country_860  country_887  
0            0            0            0  
1           

In [9]:
# Country dummy variables
filter_col = [col for col in wvs_country_df if col.startswith('country')]
print("The country dummy varibales\n")
print(filter_col)
print("\nNumber of country dummy variables : ",len(filter_col))

# Dropping one country dummy variable
wvs_country_df  = wvs_country_df.drop('country_12', axis =1)

filter_col = [col for col in wvs_country_df if col.startswith('country')]
print("\nThe country dummy varibales after removing one variable\n")
print(filter_col)
print("\nNumber of country dummy variables : ",len(filter_col))
print(len(filter_col))

The country dummy varibales

['country_12', 'country_31', 'country_32', 'country_36', 'country_48', 'country_51', 'country_76', 'country_112', 'country_152', 'country_156', 'country_158', 'country_170', 'country_196', 'country_218', 'country_233', 'country_268', 'country_275', 'country_276', 'country_288', 'country_344', 'country_356', 'country_368', 'country_392', 'country_398', 'country_400', 'country_410', 'country_417', 'country_422', 'country_434', 'country_458', 'country_484', 'country_504', 'country_528', 'country_554', 'country_566', 'country_586', 'country_604', 'country_608', 'country_616', 'country_634', 'country_642', 'country_643', 'country_646', 'country_702', 'country_705', 'country_710', 'country_716', 'country_724', 'country_752', 'country_764', 'country_780', 'country_788', 'country_792', 'country_804', 'country_840', 'country_858', 'country_860', 'country_887']

Number of country dummy variables :  58

The country dummy varibales after removing one variable

['countr

## 2 Implement Cross-Validation (40pt)
Now it's time to write your own code that does k-fold CV. I recommend to go the following path:

(3pt) Make it as a function that takes k, the (unffitted) model, features X and the target y as arguments.
(10pt) Next, one should randomly shufflee the data. However, it is easier to generate a list of indices, and shuffle those randomly.
(25pt) Loop the following k times
(a) Select every k-th of your indices for validation data

(b) For training data, select all indices, except those that went into validation data. Hint: check out set operations

(c) Separate the data X and the target y into training/validation parts.

(d) Fit the model on training data

(e) Predict outcome on validation data

(f) Compute the resulting statistic (you may compute more than one).

(2pt) Finally, return mean of the statistics

In [10]:
# Importing libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from statistics import mean 

In [11]:
# Function for k-fold cross 
def k_fold_cross_val(k , model , x , y):
    frames = [x,y]
    # Creating a dataset with target and features
    dataset = pd.concat(frames, axis =1)
    # Getting the target column name
    response_var=y.columns
    # Getting the features column names
    predictor_var= x.columns
    # Creating lists to store statistics
    fscore=[]
    accuracy=[]
    precision =[]
    recall =[]
    
    # Randomly shuffling the data
    model_shuffled = dataset.sample(frac=1)
    
    # Splitting the data in k parts
    folds = np.array_split(dataset, k)

    for i in range(k):
        # Creating the training and testing data
        train_data = folds.copy()
        test_data = folds[i]
        del train_data[i]
        train_data = pd.concat(train_data, sort=False) 
        
        # Getting the response and predictors from training and testing data
        response_train = train_data[response_var]
        pred_train=train_data[predictor_var]
        response_test = test_data[response_var]
        pred_test =test_data[predictor_var]         

        # Fitting the model
        fitted_mod = model.fit(pred_train,response_train)
        # Predicting the model
        pred= fitted_mod.predict(pred_test)

        # Storing the statistics
        accuracy.append(accuracy_score(pred,response_test))
        fscore.append(f1_score(pred,response_test, average="binary"))
        precision.append(precision_score(pred,response_test))
        recall.append(recall_score(pred,response_test))
    
    # Creating a list and returning all the mean statistics
    res = [mean(accuracy), mean(fscore), mean(precision), mean(recall)]
    return res

## 3 Find the best model (40)
In this section your task is to nfid which model: k-NN, logistic regression, or SVM works best. You will evaluate the model performance using 5-fold cross-validation with accuracy and F-score as the metric. And unlike in all your future work, here you will use your own CV impementation. Some of the methods (k-NN, SVM) are slow to compute, so you may start with a subset of data (say, 5000 random lines only). If everything turns out fine, you increase the data size as far as your computer can go.

## 3.1 k-NN (13pt)
First, use k-NN and experiment with a few different k-s.

Q1. (2pt) Separate your training data into X (features), and y (target). Target will be the abortion variable, X are all the other features.

In [12]:
#Separate data into X (features), and y (target). Target will be the abortion variable, X are all the other features.
wvs_sample_df=pd.DataFrame(wvs_country_df.sample(10000))
y=pd.DataFrame(wvs_sample_df["abortion"]) #response variable
x=pd.DataFrame(wvs_sample_df.loc[:, wvs_sample_df.columns != 'abortion']) #predictors

Q2. (2pt) pick a k and set up the k-NN model. Use your freshly-minted CV routine to cross-validate accuracy and F-score of your k-NN model.

In [13]:
import warnings
warnings.filterwarnings('ignore')

# Taking k as 10
knn_mod1=KNeighborsClassifier(n_neighbors=10)
res_knn_mod1 = k_fold_cross_val(5,knn_mod1,x,y)
print('\nThe mean accuracy for the model is  : ', res_knn_mod1[0])
print('\nThe mean f-score for the model is : ', res_knn_mod1[1])
print('\nThe mean precision score for the model is : ', res_knn_mod1[2])
print('\nThe mean recall score for the model is : ', res_knn_mod1[3])


The mean accuracy for the model is  :  0.8079000000000001

The mean f-score for the model is :  0.6965134376364082

The mean precision score for the model is :  0.6102595805409806

The mean recall score for the model is :  0.8116744646583759


Q3. (5pt) Try a few different k-NN models (pick different k, choose to normalize/not-to-normalize your features).

In [14]:
# Taking k as 50
knn_mod2=KNeighborsClassifier(n_neighbors=50)
res_knn_mod2 = k_fold_cross_val(5,knn_mod2,x,y)
print('\nThe mean accuracy for the model is  : ', res_knn_mod2[0])
print('\nThe mean f-score for the model is : ', res_knn_mod2[1])
print('\nThe mean precision score for the model is : ', res_knn_mod2[2])
print('\nThe mean recall score for the model is : ', res_knn_mod2[3])


The mean accuracy for the model is  :  0.8099

The mean f-score for the model is :  0.700268943632546

The mean precision score for the model is :  0.6149164401604876

The mean recall score for the model is :  0.8141304666434743


In [15]:
# Taking k as 100
knn_mod3=KNeighborsClassifier(n_neighbors=100)
knn_res_mod3 = k_fold_cross_val(5,knn_mod3,x,y)
print('\nThe mean accuracy for the model is  : ', knn_res_mod3[0])
print('\nThe mean f-score for the model is : ', knn_res_mod3[1])
print('\nThe mean precision score for the model is : ', knn_res_mod3[2])
print('\nThe mean recall score for the model is : ', knn_res_mod3[3])


The mean accuracy for the model is  :  0.8069000000000001

The mean f-score for the model is :  0.6866929855801174

The mean precision score for the model is :  0.5857927162472909

The mean recall score for the model is :  0.8304242899367473


Q4. (4pt) Present the results from your best k-NN model. Note: as you are using two metrics here, you may end up with different models performing better according to different measures.

Answer 4 : I have run the model for a sample of 10000 and I tried k-NN model with different values : 10, 50 and 100.

The best model according to the accuracy and fscore both is k-NN having 50 neighbours. The accuracy of the this model is around 0.81 and the fscore is around 0.70. The mean precision is 0.61 and mean recall is 0.81, which are good. But, k-NN has low computational speed.

In [16]:
print("The statistics for best k-NN model are : \n")
print('\nThe mean accuracy for the best k-NN model is  : ', res_knn_mod2[0])
print('\nThe mean f-score for the best k-NN model is : ', res_knn_mod2[1])
print('\nThe mean precision score for the best k-NN model is : ', res_knn_mod2[2])
print('\nThe mean recall score for the best k-NN model is : ', res_knn_mod2[3])

The statistics for best k-NN model are : 


The mean accuracy for the best k-NN model is  :  0.8099

The mean f-score for the best k-NN model is :  0.700268943632546

The mean precision score for the best k-NN model is :  0.6149164401604876

The mean recall score for the best k-NN model is :  0.8141304666434743


## 3.2 Logistic Regression (9pt)

Q1.Now repeat the process above with logistic regression. As we have a myriad of features anyway, we are not going to do any feature engineering. Just a plain logistic regression.

Response: 
The mean accuracy and f score for logistic regression with our sample 10000 is 0.999 i.e around 1. When we run logistic regression for the whole dataset , we get the accuracy and f-score both as 1.

Also, logistic regression has high computational speed. We are able to sample and train all of our dataset.

In [17]:
# Running Logistic Regression Model
lr_mod1=LogisticRegression()
lr_res_mod1 = k_fold_cross_val(5,lr_mod1,x,y)
print('\nThe mean accuracy for the model is  : ', lr_res_mod1[0])
print('\nThe mean f-score for the model is : ', lr_res_mod1[1])
print('\nThe mean precision score for the model is : ', lr_res_mod1[2])
print('\nThe mean recall score for the model is : ', lr_res_mod1[3])


The mean accuracy for the model is  :  0.9999

The mean f-score for the model is :  0.9998582565556343

The mean precision score for the model is :  0.9997167138810198

The mean recall score for the model is :  1.0


In [18]:
# Taking the whole sample for logistic regression
y_log =pd.DataFrame(wvs_country_df["abortion"]) #response variable
x_log =pd.DataFrame(wvs_country_df.loc[:, wvs_sample_df.columns != 'abortion']) #predictors

# Running Logistic Regression Model
lr_mod2=LogisticRegression()
lr_res_mod2 = k_fold_cross_val(5,lr_mod2,x_log,y_log)
print('\nThe mean accuracy for the model is  : ', lr_res_mod2[0])
print('\nThe mean f-score for the model is : ', lr_res_mod2[1])
print('\nThe mean precision score for the model is : ', lr_res_mod2[2])
print('\nThe mean recall score for the model is : ', lr_res_mod2[3])


The mean accuracy for the model is  :  1.0

The mean f-score for the model is :  1.0

The mean precision score for the model is :  1.0

The mean recall score for the model is :  1.0


### 3.3 SVM (15pt)

Now repeat the process with support vector machines while choosing between a few different kernels and kernel options, such as degree for polynomial kernels.

Hint: I have mixed experience with sklearn version of SVM. I recommend to limit the number of iterations, initially maybe to just 1000, in order to ensure your model actually terminates.

Q1. (14pt) pick a kernel and repeat the process above. Note that some kernels are slower than others, so be careful.

Answer 1 : For SVM, I have run the model for a sample of 10000 and used kernal linear and poly with 8 degree. 

For SVM model with linear kernal , the mean accuracy and the mean fscore both are one. The precision and recall are too 1.

For SVM model with poly kernal and degree 8 , the mean accuracy is 0.97 and mean fscore is 0.96. The mean precision score for this model is 0.94 and mean recall is 0.96.

We can see that the accuracy and fscore are high , but the SVM model has low computational speed.

In [19]:
# SVC model with kernal linear
svc_mod1 =SVC(kernel="linear")
svm_res_mod = k_fold_cross_val(5,svc_mod1,x,y)

print('\nThe mean accuracy for the model is  : ', svm_res_mod[0])
print('\nThe mean f-score for the model is : ', svm_res_mod[1])
print('\nThe mean precision score for the model is : ', svm_res_mod[2])
print('\nThe mean recall score for the model is : ', svm_res_mod[3])


The mean accuracy for the model is  :  1.0

The mean f-score for the model is :  1.0

The mean precision score for the model is :  1.0

The mean recall score for the model is :  1.0


In [20]:
# SVC model with kernal poly
svm_mod2 = SVC(kernel='poly', degree=8)

svm_res_mod2 = k_fold_cross_val(5,svm_mod2,x,y)

print('\nThe mean accuracy for the model is  : ', svm_res_mod2[0])
print('\nThe mean f-score for the model is : ', svm_res_mod2[1])
print('\nThe mean precision score for the model is : ', svm_res_mod2[2])
print('\nThe mean recall score for the model is : ', svm_res_mod2[3])


The mean accuracy for the model is  :  0.9701

The mean f-score for the model is :  0.958212415442607

The mean precision score for the model is :  0.9489339840751684

The mean recall score for the model is :  0.9677406630926889


Q2. (2pt) If your models worked like mine, you may have noticed that while accuracy seems all right, precision and recall are rather low. Explain what does such a phenomenon mean.

Answer 2: 

Accuracy is the fraction of predictions our model got right. It is the rato of right predictions to the total number of predictions. Accuarcy is a good metric to consider when the dataset is balanced i.e  we have same samples for each case. But, in many real-life problems, most datasets  have imbalanced data it becomes necessary to consider other metrics like precision and recall score.

In our case, a false positive would mean having wrongly classified a person who thinks abortion is not justifiable as a person who thinks abortion is justifiable. A false negative would mean having classified a person who thinks abortion is justifiable as a person who thinks abortion is not justifiable.

Precision is defined as the number of true positives divided by the number of true positives plus the number of false  positives (i.e the total positives identified by the model). The precision score expresses the proportion of the data points our model says was relevant actually were relevant. 

Recall is defined as the number of true positives divided by the number of true positives plus the number of false negatives.It is the ability of a model to find all the relevant cases within a dataset.

Having a high accuracy and low precision and recall score means that very few positives are actually true and also that most of the times our true poistive values are never predicted. A good model should try to minimize the false positives and false negatives along with correctly classifying data. Thus, a model with high accuracy and low precision and recall  is not ideal model, especially if the cost of the false positives and false negatives classifications are high.


### 3.4 Compare the models (3pt)

Q1. (2pt) Finally, compare the models. Which ones performed the best in terms of accuracy? Which ones in terms of F-score? Did you encounter other kind of issues with certain models? Which models were fast and which ones slow?

Answer 1 : The logstic regression and the SVM with linear kernal has the best mean accuracy and best mean f-score with value 1. K-NN has lower accuracy and fscore. The SVC with poly kernal and degree 8 has a good accuracy of 97 and fscore of 96. The logical regression has high computational speed and was fast. I was able to run the regression model on whole dataset as well in a decent time. K-NN and SVM were slow and i ran the model only on the sample of 10000, which was also very slow. I did not face any other issues except the computation time.

Q2. (1pt) If you have to repeat the exercise with a single model (and you have, see below), which one will you pick?

Answer 2 : If I had to repeat the exercise with a single model, I would select logistic regression for this dataset, as it gives us higher accuracy and fscore of 1. The precision and recall is also good for the logistic regression model. Plus, it has high computational speed and it is fast. Also, I could run my logistic model on sample as well as my whole dataset.

# 4 How large a role does country play? (20pt)
Here we switch from machine learning to social sciences. Public opinion differs from country to country, but also inside the countries. Does the fact that we include country code in data help us to substantially
improve the predictions?
You pick the best ML method from above. You estimate two sets of models: one with country information included, and one where it is removed. Is the former noticeably better than the latter?


Q1. (10pt) Pick your best ML method based you designed above. Cross-validate the accuracy of abortion variable using all the features, including country dummies and report the accuracy. Essentially you repeat here what you did above, so you can also just copy the result from above

Answer 1 : For me, the best ML methot for this dataset is Logistic Regression,  as it gives us higher accuracy and fscore  of 1 for this dataset. The precision and recall is also good for the model. Plus, it has high computational speed and it is fast. Also, I could run my logistic model on sample of 10000 as well as my whole dataset.

The mean accuracy and f score for logistic regression with our sample 10000 is 0.999 i.e around 1. When we run logistic regression for the whole dataset , we get the accuracy and f-score both as 1. The precision and recall score for the model is also 1.

Also, logistic regression has high computational speed. We are able to sample and train all of our dataset.

The exact results are show below:

In [22]:
print('The results for logistic Regression model with sample of 10000 are :')
print('\nThe mean accuracy for logistic Regression model with sample of 10000 is  : ', lr_res_mod1[0])
print('\nThe mean f-score for logistic Regression model with sample of 10000 : ', lr_res_mod1[1])
print('\nThe mean precision score for logistic Regression model with sample of 10000 is : ', lr_res_mod1[2])
print('\nThe mean recall score for logistic Regression model with sample of 10000 is : ', lr_res_mod1[3])

print('\nThe results for logistic Regression model with whole dataset are : ')
print('\nThe mean accuracy logistic Regression model with whole dataset is  : ', lr_res_mod2[0])
print('\nThe mean f-score for logistic Regression model with whole dataset is : ', lr_res_mod2[1])
print('\nThe mean precision score for logistic Regression model with whole dataset is : ', lr_res_mod2[2])
print('\nThe mean recall score for logistic Regression model with whole dataset is : ', lr_res_mod2[3])

The results for logistic Regression model with sample of 10000 are :

The mean accuracy for logistic Regression model with sample of 10000 is  :  0.9999

The mean f-score for logistic Regression model with sample of 10000 :  0.9998582565556343

The mean precision score for logistic Regression model with sample of 10000 is :  0.9997167138810198

The mean recall score for logistic Regression model with sample of 10000 is :  1.0

The results for logistic Regression model with whole dataset are : 

The mean accuracy logistic Regression model with whole dataset is  :  1.0

The mean f-score for logistic Regression model with whole dataset is :  1.0

The mean precision score for logistic Regression model with whole dataset is :  1.0

The mean recall score for logistic Regression model with whole dataset is :  1.0


Q2. (15pt) Now remove all the country dummies, but keep the other variables intact. And repeat.

In [23]:
# Removing the country dummy variables
wvs_wo_country_df = wvs_country_df.drop(filter_col, axis =1)
wvs_wo_country_df.columns

# Taking the sample for logistic regression without country
y_country =pd.DataFrame(wvs_wo_country_df["abortion"]) #response variable
x_country =pd.DataFrame(wvs_wo_country_df.loc[:, wvs_wo_country_df.columns != 'abortion']) #predictors

# Running Logistic Regression Model
lr_mod3=LogisticRegression()
lr_res_mod3 = k_fold_cross_val(5,lr_mod3,x_country,y_country)
print('The statistics for the logistic model without country variables are : ')
print('\nThe mean accuracy for the model is  : ', lr_res_mod3[0])
print('\nThe mean f-score for the model is : ', lr_res_mod3[1])
print('\nThe mean precision score for the model is : ', lr_res_mod3[2])
print('\nThe mean recall score for the model is : ', lr_res_mod3[3])


The statistics for the logistic model without country variables are : 

The mean accuracy for the model is  :  1.0

The mean f-score for the model is :  1.0

The mean precision score for the model is :  1.0

The mean recall score for the model is :  1.0


Logistic model gave accuracy 1 earlier. Removing the country variables have not affected the accuracy of fcore. I will aslo try to run the SVM model with kernal poly and degree 8 on this new dataset without country.

In [24]:
wvs_wo_country_sample_df = wvs_wo_country_df.sample(n =10000)
# Taking the sample for SVM without country
y_country =pd.DataFrame(wvs_wo_country_sample_df["abortion"]) #response variable
x_country =pd.DataFrame(wvs_wo_country_sample_df.loc[:, wvs_wo_country_sample_df.columns != 'abortion']) #predictors

svm_mod3 = SVC(kernel='poly', degree=8)

svm_res_mod3 = k_fold_cross_val(5,svm_mod3,x,y)

print("The statistics for SVM model after removing country variables are : ")
print('\nThe mean accuracy for the model is  : ', svm_res_mod3[0])
print('\nThe mean f-score for the model is : ', svm_res_mod3[1])
print('\nThe mean precision score for the model is : ', svm_res_mod3[2])
print('\nThe mean recall score for the model is : ', svm_res_mod3[3])

The statistics for SVM model after removing country variables are : 

The mean accuracy for the model is  :  0.9701

The mean f-score for the model is :  0.958212415442607

The mean precision score for the model is :  0.9489339840751684

The mean recall score for the model is :  0.9677406630926889


Q3. (5pt) Comment what you found. Does country information help to noticeably improve the prediction?

As, we can see from statistics of the logistic regression model as well as the SVM model that the accuracy and the fscore was not affected by removing the country variables.Thus, we can say that the country information has not noticeably impoved the prediction.